In [1]:
from ex03 import *

In [2]:
similarities = similarity_person(critics)

In [3]:
get_recommendations(critics, "Toby", similarities)

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]